In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, accuracy_score

In [2]:
df=pd.read_csv("/content/FINAL  COMBINED.csv",)
df

,ID,Gender,Age,BMD,t score,Porosis or penia,DM,HTN,Vit D,Calcium,Fragility fractures,MENOPAUSE,HB
0,59611.0,F,75.0,0.746,2.1,PENIA,NaN,NaN,NaN,11.6,YES,YES,NaN
1,330546.0,F,57.0,0.733,2.2,PENIA,NaN,NaN,12.8,NaN,NaN,YES,NaN
2,930163.0,F,59.0,0.873,1.2,PENIA,NO,NO,43.6,9.3,NaN,YES,14.2
3,678003.0,F,69.0,0.705,2.4,PENIA,NO,YES,NaN,9,NaN,YES,12.6
4,64708.0,F,45.0,0.927,0.8,NORMAL,NO,NO,19.4,9.3,NaN,NO,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1282,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1283,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Missing Data Analysis
missing_percentage = (df.isnull().mean()) * 100

print("Missing Data Analysis (Percentage):")
for column, percentage in missing_percentage.items():
    print(f"{column}: {percentage:.2f} ")

Missing Data Analysis (Percentage):
ID: 65.40 
Gender: 65.40 
Age: 65.40 
BMD: 65.40 
t score: 65.40 
Porosis or penia: 65.63 
DM: 66.80 
HTN: 66.64 
Vit D: 67.42 
Calcium: 66.33 
Fragility fractures: 80.48 
MENOPAUSE: 89.42 
HB: 76.05 


In [4]:
#drop HB and Fragility
df.drop(columns=['HB'], inplace=True)
df.drop(columns=['Fragility fractures'], inplace=True)

In [5]:
#Gender
df['Gender'] = df['Gender'].replace({"female ":"F","female":"F","f":"F","male":"M","m":"M"})

#Diagnosis
df.rename(columns={'Porosis or penia': 'Diagnosis'}, inplace=True)

df['Diagnosis'] = df['Diagnosis'].replace({"PENIA":"Penia","osteopenia":"Penia","osteopenia ":"Penia","Ostepenia":"Penia","Ostepenia ":'Penia',
                                           "penia":"Penia",'Osteopenia ':'Penia','Osteopenia':'Penia',"NORMAL":'Normal',"normal":'Normal','normal bmd ':'Normal','normal  bmd':'Normal',
                                           'normal bmd':'Normal','bmd within range':'Normal','POROSIS':'Porosis','osteoporosis':'Porosis','osteporosis':'Porosis','porosis':'Porosis',"porsosis":"Porosis","penia)":"Penia"})

#Hypertension
df['HTN'] = df['HTN'].replace({"NO":"No","no":"No","YES":"Yes","yes":"Yes",'htn':'Yes','htn ':'Yes',"Htn":'Yes',"yed":'Yes',"yea":"Yes"})

#Menopause
df.rename(columns={'MENOPAUSE': 'Menopause'}, inplace=True)
df['Menopause'] = df['Menopause'].replace({"NO":"No","no":"No","YES":"Yes","yes":"Yes"})

#DM
df["DM"].replace({'NO':"No","no":"No","YES":"Yes","yes":"Yes","dm":"Yes",'dm ':"Yes",
                  'PRE DM':"Pre",'pre-dm ':"Pre",'pre dm':"Pre",'pre dm ':"Pre",'Pre DM':"Pre",
                  "Pre dm":"Pre",'pre DM':"Pre","pre":"Pre",'pre-dm':'Pre','predm':'Pre'}, inplace=True)

#T-Score
df.rename(columns={'t score': 'T Score'}, inplace=True)
def convert_format(s):
    s=str(s)
    if s.endswith('-'):
        return '-' + s[:-1]
    else:
        return s
df['T Score'] = df['T Score'].apply(convert_format)


df['ID'] = df['ID'].astype('str')
df['Calcium'] = df['Calcium'].str.replace(',', '').astype(float)
df['T Score'] = df['T Score'].astype(float)

In [6]:
#Missing data filling

#MENOPAUSE

#For males
df.loc[df['Gender'] == 'M', 'Menopause'] = 'Not Applicable'

#For Females
for index, row in df.iterrows():
    if pd.isnull(row['Menopause']):
        if row['Age'] >= 52:
            df.at[index, 'Menopause'] = 'Yes'  # Assume menopause for individuals aged 50 or older
        else:
            df.at[index, 'Menopause'] = 'Unknown'  # Assume not in menopause for individuals younger than 50

#DM
df['DM'].fillna('Unavailable', inplace=True)
#HTN
df['HTN'].fillna('Unavailable', inplace=True)

#Diagnosis
df.dropna(subset=['Diagnosis'], inplace=True)

In [7]:
#Outliers cleaning
def clean_outliers(df):
    cleaned_df = df.copy()
    for column in df.select_dtypes(include='number').columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
        cleaned_df = cleaned_df[~cleaned_df.index.isin(outliers.index)]
    return cleaned_df

# Clean outliers from all numerical columns
cleaned_df = clean_outliers(df)

print("\nCleaned DataFrame:")
cleaned_df


Cleaned DataFrame:


,ID,Gender,Age,BMD,T Score,Diagnosis,DM,HTN,Vit D,Calcium,Menopause
1,330546.0,F,57.0,0.733,2.2,Penia,Unavailable,Unavailable,12.8,NaN,Yes
2,930163.0,F,59.0,0.873,1.2,Penia,No,No,43.6,9.30,Yes
3,678003.0,F,69.0,0.705,2.4,Penia,No,Yes,NaN,9.00,Yes
4,64708.0,F,45.0,0.927,0.8,Normal,No,No,19.4,9.30,No
5,854422.0,F,57.0,0.849,1.4,Penia,No,No,47.3,9.86,Yes
...,...,...,...,...,...,...,...,...,...,...,...
438,1071560.0,F,62.0,1.092,0.7,Penia,Yes,Yes,22.1,8.75,Yes
440,848225.0,F,69.0,0.929,-0.6,Porosis,No,No,68.4,9.72,Yes
441,452875.0,F,59.0,0.896,-0.9,Penia,No,Yes,10.5,9.50,Yes
443,209286.0,F,53.0,1.048,0.3,Normal,No,No,37.7,9.91,Yes


In [8]:
#Missing Data Analysis
missing_percentage = (cleaned_df.isnull().mean()) * 100

print("Missing Data Analysis (Percentage):")
for column, percentage in missing_percentage.items():
    print(f"{column}: {percentage:.2f} ")

Missing Data Analysis (Percentage):
ID: 0.00 
Gender: 0.00 
Age: 0.00 
BMD: 0.00 
T Score: 0.00 
Diagnosis: 0.00 
DM: 0.00 
HTN: 0.00 
Vit D: 5.13 
Calcium: 2.93 
Menopause: 0.00 


In [9]:
cleaned_df.dtypes

ID            object
Gender        object
Age          float64
BMD          float64
T Score      float64
Diagnosis     object
DM            object
HTN           object
Vit D        float64
Calcium      float64
Menopause     object
dtype: object

In [10]:
cleaned_df

,ID,Gender,Age,BMD,T Score,Diagnosis,DM,HTN,Vit D,Calcium,Menopause
1,330546.0,F,57.0,0.733,2.2,Penia,Unavailable,Unavailable,12.8,NaN,Yes
2,930163.0,F,59.0,0.873,1.2,Penia,No,No,43.6,9.30,Yes
3,678003.0,F,69.0,0.705,2.4,Penia,No,Yes,NaN,9.00,Yes
4,64708.0,F,45.0,0.927,0.8,Normal,No,No,19.4,9.30,No
5,854422.0,F,57.0,0.849,1.4,Penia,No,No,47.3,9.86,Yes
...,...,...,...,...,...,...,...,...,...,...,...
438,1071560.0,F,62.0,1.092,0.7,Penia,Yes,Yes,22.1,8.75,Yes
440,848225.0,F,69.0,0.929,-0.6,Porosis,No,No,68.4,9.72,Yes
441,452875.0,F,59.0,0.896,-0.9,Penia,No,Yes,10.5,9.50,Yes
443,209286.0,F,53.0,1.048,0.3,Normal,No,No,37.7,9.91,Yes


In [11]:
# Save the cleaned DataFrame to a CSV file
cleaned_df.to_csv("Final_Cleaned.csv", index=False)
